In [ ]:
# Example of how to use Ovito modifiers from Neomd
# and update the database with new attributes

In [6]:
import neomd
from neomd import calculator, converter
import neomd.querybuilder
import neo4j
from ovito.pipeline import StaticSource, Pipeline
from ovito.io.ase import ase_to_ovito
import ovito

driver = neo4j.GraphDatabase.driver("bolt://127.0.0.1:7687", auth=("neo4j", "secret"))

qb = neomd.querybuilder.Neo4jQueryBuilder([('State', 'nano_pt', 'State', 'ONE-TO-ONE'),
                                           ('Atom', 'PART_OF', 'State', 'MANY-TO-ONE')])

q = qb.generate_trajectory('nano_pt',
                           "ASC", ['RELATION', 'timestep'],
                            node_attributes=[],
                            relation_attributes=[],
                            include_atoms=True)        
            
state_atom_dict = converter.query_to_ASE(driver, qb, q, 'Pt', False)

In [7]:
new_attributes = calculator.apply_ovito_pipeline_modifier(state_atom_dict, analysisType='AcklandJonesModifier')

In [8]:
q = None
with driver.session() as session:
    tx = session.begin_transaction()
    for state_number, data in new_attributes.items():
        if q is None:
            q = qb.generate_update_entity(data, 
                              'State', 
                              'number', 
                              'NODE')
        data.update({'number': state_number})
        tx.run(q.text, data)
    tx.commit()

In [9]:
with driver.session() as session:
    res = session.run("MATCH (n:State)-[:nano_pt]->() RETURN n limit 10;")
    for r in res:
        print(r)

<Record n=<Node id=3344 labels=frozenset({'State'}) properties={'boxabc_z_z': 0.0, 'occurrences': 6, 'boxabc_z_y': 1.32870774e-316, 'xy': 0.0, 'xz': 0.0, 'boxabc_x_x': 1.3287504e-316, 'AcklandJones_counts_ICO': 0, 'boxabc_z_x': 6.95333658158897e-310, 'boxabc_x_z': 1.32870774e-316, 'boxabc_x_y': 1.3287504e-316, 'number': '17758426787395080545', 'origin_y': 0.0, 'origin_z': 0.0, 'rsp_z_x': 0.0, 'rsp_x_y': 0.0, 'rsp_x_z': 0.0, 'rsp_x_x': 30.0, 'rsp_z_z': 30.0, 'rsp_z_y': 0.0, 'yz': 0.0, 'periodic_y': 1.0, 'periodic_x': 1.0, 'periodic_z': 1.0, 'csp_z_y': 0.0, 'csp_z_x': 0.0, 'csp_x_z': 0.0, 'csp_z_z': 0.03333333333333333, 'origin_x': 0.0, 'csp_x_y': 0.0, 'csp_x_x': 0.03333333333333333, 'AtomCount': 147, 'boxabc_y_z': 2.69426e-317, 'AcklandJones_counts_HCP': 33, 'boxabc_y_y': 6.95333658156763e-310, 'boxabc_y_x': 1.32870774e-316, 'boxlo_y': 0.0, 'AcklandJones_counts_BCC': 3, 'boxlo_z': 0.0, 'AcklandJones_counts_FCC': 6, 'rsp_y_x': 0.0, 'rsp_y_y': 30.0, 'boxhi_x': 30.0, 'boxlo_x': 0.0, 'boxhi

In [ ]:
# How to get all of the atom counts for all states

In [3]:
import neomd
import neomd.querybuilder
import neo4j

driver = neo4j.GraphDatabase.driver("bolt://127.0.0.1:7687", auth=("neo4j", "secret"))

qb = neomd.querybuilder.Neo4jQueryBuilder([('State', 'nano_pt', 'State', 'ONE-TO-ONE'),
                                           ('Atom', 'PART_OF', 'State', 'MANY-TO-ONE')])

q = qb.generate_calculate_many_to_one_count("PART_OF")

q.text

'MATCH (n1:Atom)-[r:PART_OF]->(n2:State) WITH n2, count(r) as AtomCount SET n2.AtomCount = AtomCount;'

In [4]:
with driver.session() as session:
    session.run(q.text)
    result = session.run("MATCH (n:State)-[:nano_pt]-() RETURN n LIMIT 10;")
    for r in result:
        print(r)

<Record n=<Node id=3344 labels=frozenset({'State'}) properties={'boxabc_z_z': 0.0, 'boxabc_z_y': 1.32870774e-316, 'boxabc_y_z': 2.69426e-317, 'xy': 0.0, 'xz': 0.0, 'boxabc_x_x': 1.3287504e-316, 'boxabc_z_x': 6.95333658158897e-310, 'boxabc_y_y': 6.95333658156763e-310, 'boxabc_x_z': 1.32870774e-316, 'boxabc_y_x': 1.32870774e-316, 'boxabc_x_y': 1.3287504e-316, 'number': '17758426787395080545', 'boxlo_y': 0.0, 'origin_y': 0.0, 'boxlo_z': 0.0, 'origin_z': 0.0, 'rsp_z_x': 0.0, 'rsp_y_x': 0.0, 'rsp_x_y': 0.0, 'rsp_y_y': 30.0, 'rsp_x_z': 0.0, 'rsp_x_x': 30.0, 'boxhi_x': 30.0, 'boxlo_x': 0.0, 'boxhi_y': 30.0, 'rsp_z_z': 30.0, 'rsp_y_z': 0.0, 'boxhi_z': 30.0, 'rsp_z_y': 0.0, 'yz': 0.0, 'periodic_y': 1.0, 'periodic_x': 1.0, 'periodic_z': 1.0, 'csp_z_y': 0.0, 'csp_y_z': 0.0, 'csp_z_x': 0.0, 'csp_y_y': 0.03333333333333333, 'csp_x_z': 0.0, 'csp_z_z': 0.03333333333333333, 'origin_x': 0.0, 'csp_y_x': 0.0, 'csp_x_y': 0.0, 'csp_x_x': 0.03333333333333333, 'AtomCount': 147}>>
<Record n=<Node id=3344 label

In [1]:
import neomd
import neomd.querybuilder
import neomd.visualizations
import neomd.converter
import neo4j


driver = neo4j.GraphDatabase.driver("bolt://127.0.0.1:7687", auth=("neo4j", "secret"))

qb = neomd.querybuilder.Neo4jQueryBuilder([('State', 'nano_pt', 'State', 'ONE-TO-ONE'),
                                           ('Atom', 'PART_OF', 'State', 'MANY-TO-ONE')])

q = qb.generate_get_path(300, 400, 'nano_pt', 'timestep')

state_atom_dict = neomd.converter.query_to_ASE(driver,qb,q,'Pt')

vid = neomd.visualizations.render_ASE_list(state_atom_dict)

vid

QApplication: invalid style override 'kvantum' passed, ignoring it.
	Available styles: Windows, Fusion


{'4367321625357726361': Atoms(symbols='Pt147', pbc=True, cell=[30.0, 30.0, 30.0], id=..., masses=..., momenta=..., travel=..., type=...), '11119287140548197726': Atoms(symbols='Pt147', pbc=True, cell=[30.0, 30.0, 30.0], id=..., masses=..., momenta=..., travel=..., type=...), '9013864045735191222': Atoms(symbols='Pt147', pbc=True, cell=[30.0, 30.0, 30.0], id=..., masses=..., momenta=..., travel=..., type=...), '2163921597563625916': Atoms(symbols='Pt147', pbc=True, cell=[30.0, 30.0, 30.0], id=..., masses=..., momenta=..., travel=..., type=...), '17358659149290685432': Atoms(symbols='Pt147', pbc=True, cell=[30.0, 30.0, 30.0], id=..., masses=..., momenta=..., travel=..., type=...), '6548325500823722235': Atoms(symbols='Pt147', pbc=True, cell=[30.0, 30.0, 30.0], id=..., masses=..., momenta=..., travel=..., type=...), '5463845886973532215': Atoms(symbols='Pt147', pbc=True, cell=[30.0, 30.0, 30.0], id=..., masses=..., momenta=..., travel=..., type=...), '10964846041268685003': Atoms(symbols=

'vid.mp4'

In [4]:
import neomd
from neomd import calculator, converter, querybuilder
import neo4j
import pandas as pd

# gets the sequence as a list of ids

driver = neo4j.GraphDatabase.driver("bolt://127.0.0.1:7687",
                                        auth=("neo4j", "secret"))

qb = querybuilder.Neo4jQueryBuilder(
        [('State', 'nano_pt', 'State', 'ONE-TO-ONE'),
         ('Atom', 'PART_OF', 'State', 'MANY-TO-ONE')])

q = qb.generate_trajectory("nano_pt",
                            "ASC", ('relation', 'timestep'),
                            node_attributes=[('id', 'first')])
sequence = None
with driver.session() as session:
    result = session.run(q.text)
    sequence = result.value()

In [5]:
q = qb.generate_get_all_nodes("State",
                              ["id", "occurrences", "number"],
                              relation="nano_pt")
uniqueStates = None

with driver.session() as session:
    result = session.run(q.text)
    uniqueStates = result.data()

[{'id': 1, 'occurrences': 31, 'number': '17758426787395080545'},
 {'id': 2, 'occurrences': 15, 'number': '16668995811752463442'},
 {'id': 3, 'occurrences': 9, 'number': '15066553743947333739'},
 {'id': 4, 'occurrences': 9, 'number': '18402056612274874415'},
 {'id': 5, 'occurrences': 8, 'number': '4367321625357726361'},
 {'id': 6, 'occurrences': 32, 'number': '11119287140548197726'},
 {'id': 7, 'occurrences': 9, 'number': '10207727307949282287'},
 {'id': 8, 'occurrences': 1, 'number': '9779374900956576290'},
 {'id': 9, 'occurrences': 1, 'number': '11482371922892328063'},
 {'id': 10, 'occurrences': 18, 'number': '3155575347548057479'},
 {'id': 11, 'occurrences': 9, 'number': '7726805181963342055'},
 {'id': 12, 'occurrences': 3, 'number': '13849239979223784777'},
 {'id': 13, 'occurrences': 3, 'number': '10186254592632817022'},
 {'id': 14, 'occurrences': 4, 'number': '5348500487128154003'},
 {'id': 15, 'occurrences': 3, 'number': '16725342732876878191'},
 {'id': 16, 'occurrences': 3, 'numb

In [8]:
m, idx_to_id = neomd.calculator.calculate_transition_matrix(driver, 
qb, run='nano_pt', 
discrete=True, 
trajectory=sequence, uniqueStates=uniqueStates)

In [9]:
m

,1,2,3,4,5,6,7,8,9,10,...,2584,2585,2586,2587,2588,2589,2590,2591,2592,2593
0,0.0,0.483871,0.0,0.0,0.225806,0.000,0.0,0.032258,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.4,0.000000,0.6,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,1.0,0.000000,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.000000,0.875,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2588,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2589,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2590,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2591,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
